In [1]:
!pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 12.2 MB/s eta 0:00:00


In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
print(tf.__version__)

2.19.0


In [3]:
fashion_mnist=keras.datasets.fashion_mnist

In [4]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [5]:
train_images=train_images/255.0
test_images=test_images/255.0

In [6]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [7]:
def build_model(hp):
  model=keras.Sequential([
      keras.layers.Input(shape=(28,28,1)),
      keras.layers.Conv2D(
          filters=hp.Int('conv_1_filter',min_value=32,max_value=128,step=16),
          kernel_size=hp.Choice('conv_1_kernel',values=[3,5]),
          activation='relu'
      ),
      keras.layers.MaxPooling2D(pool_size=(2,2)),
      keras.layers.Conv2D(
          filters=hp.Int('conv_2_filter',min_value=32,max_value=64,step=16),
          kernel_size=hp.Choice('conv_2_kernel',values=[3,5]),
          activation='relu'
      ),
      keras.layers.MaxPooling2D(pool_size=(2,2)),
      keras.layers.Flatten(),
      keras.layers.Dense(
          units=hp.Int('Dense_units',min_value=32,max_value=128,step=16),
          activation='relu'
      ),
      keras.layers.Dense(10,activation='softmax')
      ])
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-3,1e-2])),loss='sparse_categorical_crossentropy',metrics=['accuracy'])
  return model

In [8]:
from keras_tuner import RandomSearch

In [9]:
tuner = RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory='output',project_name='mnist_fashion')

In [10]:
tuner.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 24s]
val_accuracy: 0.8711666464805603

Best val_accuracy So Far: 0.9023333191871643
Total elapsed time: 00h 02m 47s


In [11]:
model=tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [12]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 26, 26, 48)     │           480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 13, 13, 48)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 11, 11, 64)     │        27,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1600)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 96)             │       153,696 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │           970 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 182,858 (714.29 KB)

 Trainable params: 182,858 (714.29 KB)

 Non-trainable params: 0 (0.00 B)

In [13]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)
if best_hps:
    print(best_hps[0].values)
else:
    print("No successful trials found")

{'conv_1_filter': 48, 'conv_1_kernel': 3, 'conv_2_filter': 64, 'conv_2_kernel': 3, 'Dense_units': 96, 'learning_rate': 0.001}


In [14]:
model.fit(train_images,train_labels,epochs=10,initial_epoch=3,validation_data=(test_images,test_labels))

Epoch 4/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 12s 5ms/step - accuracy: 0.9099 - loss: 0.2387 - val_accuracy: 0.9057 - val_loss: 0.2603
Epoch 5/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9246 - loss: 0.2045 - val_accuracy: 0.9088 - val_loss: 0.2512
Epoch 6/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9327 - loss: 0.1819 - val_accuracy: 0.9045 - val_loss: 0.2663
Epoch 7/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.9383 - loss: 0.1650 - val_accuracy: 0.9040 - val_loss: 0.2836
Epoch 8/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.9447 - loss: 0.1432 - val_accuracy: 0.9051 - val_loss: 0.2849
Epoch 9/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9529 - loss: 0.1266 - val_accuracy: 0.9093 - val_loss: 0.2776
Epoch 10/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 12s 5ms/step - accuracy: 0.9586 - loss: 0.1087 - val_accuracy: 0.9112 - val_loss: 0.2819


In [15]:
keras.models.save_model('optimizer.keras')

TypeError: save_model() missing 1 required positional argument: 'filepath'